# Creating a New Thorn -- HeatEqn
---

In this tutorial we will walk through the process of creating a new thorn from scratch. As a new user, you may be content at first to run simulations that have already been assembled by someone else (such as the gallery examples). Beyond that you may want to assemble your own simulation using existing thorns. However at some point you may have an application that involves a step, or calculation, or algorithm, that no existing thorn performs. At that point, you need to write your own thorn to carry out that subroutine. The case study we will consider here is to create a thorn to evolve the standard heat equation.

Let's start with a brief overview of what a thorn needs to include:

* a Cactus thorn must have a name.
* a Cactus thorn must live in an arrangement.
* a Cactus thorn must have four ccl (Cactus Configuration Language) files:
    * `interface.ccl`
    * `schedule.ccl`
    * `param.ccl`
    * `configuration.ccl`
* a Cactus thorn must have a `src` directory
* a Cactus thorn must have a `make.code.defn` file in that source directory

The above thorn files and directories should reside in `Cactus/arrangements/ArrangementName/ThornName/`.

In [ ]:
# this allows you to use "cd" in cells to change directories instead of requiring "%cd"
%automagic on
# override IPython's default %%bash to not buffer all output
from IPython.core.magic import register_cell_magic
@register_cell_magic
def bash(line, cell): get_ipython().system(cell)

## 0. Preliminaries

First let's create a variable for the Cactus directory on your machine and then move to that directory (in case it is not already the working directory).

In [ ]:
## Linux example:
#cactus_dir='/home/ejwest/ETK/Cactus'

## Windows example:
#cactus_dir='C:\\Users\\EJWest\\ETK\\Cactus'
import os
home_dir = os.environ["HOME"]

## fill in the path to your Cactus directory
cactus_dir=os.path.join(home_dir,'Cactus') #<-- COMPLETE THIS LINE!

In [ ]:
%cd {cactus_dir}

Next let's specify the name and arrangement of the thorn we want to make. Here we are creating a thorn to evolve the heat equation, so we will name it `HeatEqn` and place it in the arrangment `Tests`. As a result, the files we create below will be located in `Cactus/arrangements/Tests/HeatEqn`. The following block creates some environment variables that will be used below. 

In [ ]:
## Define some basic parameters describing a new thorn
thorn_pars = {
   "thorn_name" : "HeatEqn",
   "arrangement_name" : "Tests",
 }

## Define environment variables ARR and THORN to be used
## throughout the rest of the tutorial
import os
os.environ["ARR"]=thorn_pars["arrangement_name"]
os.environ["THORN"]=thorn_pars["thorn_name"]

This next command is only needed if you want to start over. Uncomment and run this line to delete the thorn you created and start over.

In [ ]:
#%rm -rf arrangements/$ARR/$THORN

Next we create the thorn directory and sub-directories.

In [ ]:
## The equivalent of mkdir -p
def create_dir(dir):
    print("Ensuring directory '"+dir+"'")
    os.makedirs(dir, exist_ok=True)
        
## Create the thorn directory inside the Cactus source tree
arrangement_dir = os.path.join(cactus_dir, "arrangements", thorn_pars["arrangement_name"])
thorn_dir = os.path.join(arrangement_dir, thorn_pars["thorn_name"])
create_dir(thorn_dir)

## Create the source directory
src_dir = os.path.join(thorn_dir, "src")
create_dir(src_dir)

## Other dirs and files not strictly needed
## for compiling and running Cactus.
test_dir = os.path.join(thorn_dir, "test")
create_dir(test_dir)
par_dir = os.path.join(thorn_dir, "par")
create_dir(par_dir)
doc_dir = os.path.join(thorn_dir, "doc")
create_dir(doc_dir)

Now move to the thorn directory that we just created and examine its contents. All that appears so far are the sub-directories that we created above.

In [ ]:
print()
print('Directory contents:')
%ls {thorn_dir}

We're finally ready to start creating the files that make up the thorn. We can write the contents of a notebook cell to a file by using the `%%writefile` command. A command that begins with `%%` is called a "cell-magic", since it operates on the entire content of a cell (as opposed to a "line-magic", prefixed by `%`, which only operates on the same line). We will use the `%%writefile` cell-magic repeatedly throughout this tutorial to create files. Of course you could also create the files outside of the notebook, using your favorite editor. Working in the notebook is just a shortcut. 

As a first application of the `%%writefile` magic, let's create a README file for our thorn. Below is the standard template for that. Change the names and email addresses to your own and then execute the cell.

In [ ]:
%%writefile {thorn_dir}/README
Author(s)    : Eric J. West <ewest@d.umn.edu>
Maintainer(s): Eric J. West <ewest@d.umn.edu>
Licence      : BSD
--------------------------------------------------------------------------

1. Purpose

Basic implementation evolving the standard 3D heat equation.

Now re-examine the contents of the thorn directory. The README file we just created should be listed.

In [ ]:
%ls {thorn_dir}

## 1/2. Heat Equation

The equation we want to solve is the standard 3D heat equation. Let's call the dependent variable $U(t,x,y,z)$. In terms of Cartesian coordinates, the heat equation is written as

\begin{align}
  \partial_{t}U
  = \partial_{x}^{2}U + \partial_{y}^{2}U + \partial_{z}^{2}U,
\end{align}

where units have been chosen so that the diffusion coefficient $\kappa=1$. Applying a simple foward-time centered-space finite-difference scheme gives

\begin{align}
  U^{n+1}_{i,j,k} 
  = U^{n}_{i,j,k} 
  + \Delta{t}\left[
    \frac{U^{n}_{i+1,j,k} + U^{n}_{i-1,j,k} - 2U^{n}_{i,j,k}}{(\Delta{x})^{2}} 
  + \frac{U^{n}_{i,j+1,k} + U^{n}_{i,j-1,k} - 2U^{n}_{i,j,k}}{(\Delta{y})^{2}} 
  + \frac{U^{n}_{i,j,k+1} + U^{n}_{i,j,k-1} - 2U^{n}_{i,j,k}}{(\Delta{z})^{2}}\right]
\end{align}

For simplicity we will seek solutions that vanish at the boundaries

\begin{align}
  & U(t,0,y,z) = U(t,L_{x},y,z) = 0 \\
  & U(t,x,0,z) = U(t,x,L_{y},z) = 0 \\
  & U(t,x,y,0) = U(t,x,y,L_{z}) = 0
\end{align}

We will take the initial configuration to be a Gaussian

\begin{align}
  U(0,x,y,z) = e^{-(x^2 + y^2 + z^2)}
\end{align}

The equations, boundary conditions, and initial condition are implemented in the following source code files.

## 1. CCL files



First, we will create the four required CCL files.

In the `interface.ccl` file, we can specify which thorns we want to inherit definitions from. We can also define our own variables which are visible to our source code and other thorns.

Variables declared in `interface.ccl` are arranged in *variable groups*. Variables in the same group share common properties. In this case, we want to define 1 variable, `U`.

In [ ]:
%%writefile {thorn_dir}/interface.ccl
## Interface definitions for thorn HeatEqn

## Inherit definitions from the Grid thorn.
## Namely, the x, y, and z variables.
inherits: Grid

## An implementation name is required for all thorns. No
## two thorns in a configuration can implement the same
## interface. This is essentially the name of our thorn.
implements: HeatEqn

## The groups declared below can be public, private, or protected.
public:

## The following syntax declares a variable group called evol_group with one variable, U.
## As specified by TYPE=GF, U is a Grid Function (distributed 3d array) whose elements are of type CCTK_REAL.
## In our source files, we will need to access both the current values of U, as well as the values from the
## previous time step. To accomplish this, we set TIMELEVELS=2. This implicitly declares U_p (U at the
## previous time step) in addition to U. TIMELEVELS=3 would also declare U_p_p, and so forth. 
CCTK_REAL evol_group TYPE=GF TIMELEVELS=2
{
  U
} "Heat equation fields"

## Scalars are single variables that are available on all processors.
#CCTK_REAL scalar_group TYPE=SCALAR 
#{
#  scalar1, scalar2
#}

The `schedule.ccl` file is where you declare storage allocations as well as telling Cactus when to execute thorn functions. Cactus schedules its work in several bins. While there are many of them, here we only consider two:

1) CCTK_INITIAL - This runs once at the beginning. Initialize your grid functions here.

2) CCTK_EVOL - Evolve a single timestep forward. This step will run repeatedly until the simulation finishes. For the heat equation, the Euler method is sufficient to evolve the system. If thorns require a more sophisticated time-stepping algorithm, such as RK4, they should instead use the MOL (Method of Lines) thorn. See the tutorial, "Creating a new thorn Wave Equation."

We will schedule 3 functions:

- HeatEqn_Initialize, to initialize the grid function U.
- HeatEqn_Update, to evolve the grid function U one timestep forward.
- HeatEqn_Boundary, to handle the boundary conditions.

And we will allocate storage for our one and only group:

- evol_group

The next cell generates the `schedule.ccl` file.

In [ ]:
%%writefile {thorn_dir}/schedule.ccl
## Schedule definitions for thorn HeatEqn

## There won't be any storage allocated for a group
## unless a corresponding storage declaration exists
## for it in the schedule file. In square brackets,
## we specify the number of storage levels to allocate.
## Since we set TIMELEVELS=2, we should allocate 2 levels.
STORAGE: evol_group[2]

## Schedule a function defined in this thorn to run at various stages
## of the simulation. The minimum you need to specify for a schedule
## item is what language it's written in. Choices are: C (which includes
## C++) and Fortran (which means Fortran90).

## In the declarations below, we also include Writes and Reads properties. These specify which variables,
## and which zone (Everywhere, Interior, or Boundary) of each variable, each function accesses. These
## declarations help Carpet figure out how to update the ghost zones.

SCHEDULE HeatEqn_Initialize AT CCTK_INITIAL
{
   LANG: C
   Writes: U(Everywhere)
   Reads: Grid::coordinates(Everywhere) ## `coordinates` is from a different thorn, Grid, so we need
                                        ## to specify the latter explicitly.
} "Initialize evolved variables"

SCHEDULE HeatEqn_Update AT CCTK_EVOL
{
   LANG: C
   Writes: U(Interior)
   Reads: U_p(Everywhere)
} "Evolve the Heat equation"

SCHEDULE HeatEqn_Boundary AT CCTK_EVOL AFTER HeatEqn_Update
{
   LANG: C
   Writes: U(Boundary)
   SYNC: evol_group ## SYNC tells Carpet to update the ghost zones for a variable group.
                    ## Ghost zones and boundaries are handled separately.
} "Heat equation BC"

The `param.ccl` file is where you declare runtime parameters. In doing so, you specify the allowed values, types, and defaults of those parameters. The parameters declared here are assignable at runtime from a parfile. In our example, we have no parameters, so we leave this file empty (except for comments that you may find useful for future reference).

In [ ]:
%%writefile {thorn_dir}/param.ccl
## Parameter definitions for thorn HeatEqn

## There are five types of parameters: INT, REAL, KEYWORD, STRING, 
## and BOOLEAN. The following comments provide prototypes of each.
#
#CCTK_INT one_to_five "This integer parameter goes from 1 to 5"
#{
#  1:5 :: "Another comment"
#} 3 # This is the default value
#
#CCTK_REAL from_2p5_to_3p8e4 "This integer parameter goes from 2.5 to 3.8e4"
#{
#  2.5:3.8e4 :: "Another comment"
#} 4.4e3 # This is the default value
#
## This keyword example defines the parameter wavemaker_type and 8 possible values.
#CCTK_KEYWORD wavemaker_type "types of wave makers"
#{
#  "ini_rec" :: "initial rectangular hump, need xc,yc and wid"
#  "lef_sol" :: "initial solitary wave, WKN B solution, need amp, dep"
#  "ini_oth" :: "other initial distribution specified by users"
#  "ini_gau" :: "initial Gaussian hump, need amp, xc, yc, and wid"
#  "ini_sol" :: "initial solitary wave, xwavemaker"
#} "ini_gau" # This is the default value
#
#CCTK_STRING a_string_par "a comment"
#{
#  .* :: "This is a perl 5 regular expression defining what the string may contain"
#} "blah blah blah" # This is the default value
#
#BOOLEAN a_boolean_par "a comment"
#{
#} true

The `configure.ccl` file is optional. See the [Users Guide](https://einsteintoolkit.org/usersguide/UsersGuide.html).

In [ ]:
%%writefile {thorn_dir}/configure.ccl
## Configuration definitions for thorn HeatEqn

## You should not need to include "mpi.h", but if you
## do, you will need this next line.
# REQUIRES MPI
# REQUIRES HDF5

## 2. Source code and make file

In this section we will create the contents of the `src` subdirectory. This includes any source code (written in either C++ or Fortran) and a make file.

The following file initializes the solution, implementing the initial conditions.

In [ ]:
%%writefile {thorn_dir}/src/init.c
#include <math.h>
#include <cctk.h>
#include <cctk_Arguments.h>
#include <cctk_Parameters.h>
#include <stdio.h>

// Initialize grid functions
void HeatEqn_Initialize(CCTK_ARGUMENTS) 
{
  DECLARE_CCTK_ARGUMENTS_HeatEqn_Initialize;  // Declare all grid functions from interface.ccl
  DECLARE_CCTK_PARAMETERS; // Declare all parameters from param.ccl

  // cctk_lsh is implicitly declared inside DECLARE_CCTK_ARGUMENTS_HeatEqn_Initialize.
  // cctk_lsh[0] is the size of the x dimension. Indices 1 and 2 are the sizes of the y and z dimensions.

  for (int k = 0; k < cctk_lsh[2]; k++) // loop over the z direction
  {
    for (int j = 0; j < cctk_lsh[1]; j++) // loop over the y direction
    {
      for (int i = 0; i < cctk_lsh[0]; i++) // loop over the x direction
      {
        // cctkGH is implicitly declared. It is an opaque value which many functions require as an argument.
        // CCTK_GFINDEX3D, one such function, computes a scalar grid function index from x, y, and z indices. 
        const size_t ijk = CCTK_GFINDEX3D(cctkGH, i, j, k);
        U[ijk] = exp(-x[ijk] * x[ijk] - y[ijk] * y[ijk] - z[ijk] * z[ijk]); // initial Gaussian
      }
    }
  }
}

The following file updates the solution at the interior grid points for each timestep.

In [ ]:
%%writefile {thorn_dir}/src/evolve.c
#include <cctk.h>
#include <cctk_Arguments.h>
#include <cctk_Parameters.h>
#include <stdio.h>

// Update grid functions at interior points
void HeatEqn_Update(CCTK_ARGUMENTS)
{
  DECLARE_CCTK_ARGUMENTS_HeatEqn_Update;  // Declare all grid functions from interface.ccl
  DECLARE_CCTK_PARAMETERS; // Declare all parameters from param.ccl

  const int gz = cctk_nghostzones[2];
  const int gy = cctk_nghostzones[1];
  const int gx = cctk_nghostzones[0];

  const CCTK_REAL dt = CCTK_DELTA_TIME;

  const CCTK_REAL dx = CCTK_DELTA_SPACE(0);
  const CCTK_REAL dy = CCTK_DELTA_SPACE(1);
  const CCTK_REAL dz = CCTK_DELTA_SPACE(2);

  for (int k = gz; k < cctk_lsh[2] - gz; k++) // loop over the z direction
  {
    for (int j = gy; j < cctk_lsh[1] - gy; j++) // loop over the y direction
    {
      for (int i = gx; i < cctk_lsh[0] - gx; i++) // loop over the x direction
      {
        const size_t ijk = CCTK_GFINDEX3D(cctkGH, i, j, k);
        const size_t ip1jk = CCTK_GFINDEX3D(cctkGH, i + 1, j, k);
        const size_t im1jk = CCTK_GFINDEX3D(cctkGH, i - 1, j, k);
        const size_t ijp1k = CCTK_GFINDEX3D(cctkGH, i, j + 1, k);
        const size_t ijm1k = CCTK_GFINDEX3D(cctkGH, i, j - 1, k);
        const size_t ijkp1 = CCTK_GFINDEX3D(cctkGH, i, j, k + 1);
        const size_t ijkm1 = CCTK_GFINDEX3D(cctkGH, i, j, k - 1);

        // U_p is the value of U at the previous time step. We have access to this because we declared
        // evol_group to have TIMELEVELS=2.

        const CCTK_REAL laplacian_U = (U_p[ip1jk] + U_p[im1jk] - 2 * U_p[ijk]) / (dx * dx)
                                    + (U_p[ijp1k] + U_p[ijm1k] - 2 * U_p[ijk]) / (dy * dy)
                                    + (U_p[ijkp1] + U_p[ijkm1] - 2 * U_p[ijk]) / (dz * dz);

        U[ijk] = U_p[ijk] + dt * laplacian_U;
      }
    }
  }
}

The following file imposes the boundary conditions at each timestep. This should be done after the interior points are updated.

In [ ]:
%%writefile {thorn_dir}/src/boundary.c
#include <cctk.h>
#include <cctk_Arguments.h>
#include <cctk_Parameters.h>
#include <stdio.h>

// Update grid functions at boundary points
void HeatEqn_Boundary(CCTK_ARGUMENTS) 
{
  DECLARE_CCTK_ARGUMENTS_HeatEqn_Boundary;  // Declare all grid functions from interface.ccl
  DECLARE_CCTK_PARAMETERS; // Declare all parameters from param.ccl

  const int gz = cctk_nghostzones[2];
  const int gy = cctk_nghostzones[1];
  const int gx = cctk_nghostzones[0];

  const CCTK_REAL dt = CCTK_DELTA_TIME;

  // Lower X Boundary
  if (cctk_bbox[0])
  {
    for (int k = 0; k < cctk_lsh[2]; k++) // loop over the z direction
    {
      for (int j = 0; j < cctk_lsh[1]; j++) // loop over the y direction
      {
        for (int i = 0; i < gx; i++) // loop over the x direction
        {
          const size_t ijk = CCTK_GFINDEX3D(cctkGH, i, j, k);
          U[ijk] = 0.0; // Dirichlet boundary condition
        }
      }
    }
  }

  // Upper X Boundary
  if (cctk_bbox[1])
  {
    for (int k = 0; k < cctk_lsh[2]; k++) // loop over the z direction
    {
      for (int j = 0; j < cctk_lsh[1]; j++) // loop over the y direction
      {
        for (int i = cctk_lsh[0] - gx; i < cctk_lsh[0]; i++) // loop over the x direction
        {
          const size_t ijk = CCTK_GFINDEX3D(cctkGH, i, j, k);
          U[ijk] = 0.0; // Dirichlet boundary condition
        }
      }
    }
  }

  // Lower Y Boundary
  if (cctk_bbox[2])
  {
    for (int k = 0; k < cctk_lsh[2]; k++) // loop over the z direction
    {
      for (int j = 0; j < gy; j++) // loop over the y direction
      {
        for (int i = 0; i < cctk_lsh[0]; i++) // loop over the x direction
        {
          const size_t ijk = CCTK_GFINDEX3D(cctkGH, i, j, k);
          U[ijk] = 0.0; // Dirichlet boundary condition
        }
      }
    }
  }

  // Upper Y Boundary
  if (cctk_bbox[3])
  {
    for (int k = 0; k < cctk_lsh[2]; k++) // loop over the z direction
    {
      for (int j = cctk_lsh[1] - gy; j < cctk_lsh[1]; j++) // loop over the y direction
      {
        for (int i = 0; i < cctk_lsh[0]; i++) // loop over the x direction
        {
          const size_t ijk = CCTK_GFINDEX3D(cctkGH, i, j, k);
          U[ijk] = 0.0; // Dirichlet boundary condition
        }
      }
    }
  }

  // Lower Z Boundary
  if (cctk_bbox[4])
  {
    for (int k = 0; k < gz; k++) // loop over the z direction
    {
      for (int j = 0; j < cctk_lsh[1]; j++) // loop over the y direction
      {
        for (int i = 0; i < cctk_lsh[0]; i++) // loop over the x direction
        {
          const size_t ijk = CCTK_GFINDEX3D(cctkGH, i, j, k);
          U[ijk] = 0.0; // Dirichlet boundary condition
        }
      }
    }
  }

  // Upper Z Boundary
  if (cctk_bbox[5])
  {
    for (int k = cctk_lsh[2] - gz; k < cctk_lsh[2]; k++) // loop over the z direction
    {
      for (int j = 0; j < cctk_lsh[1]; j++) // loop over the y direction
      {
        for (int i = 0; i < cctk_lsh[0]; i++) // loop over the x direction
        {
          const size_t ijk = CCTK_GFINDEX3D(cctkGH, i, j, k);
          U[ijk] = 0.0; // Dirichlet boundary condition
        }
      }
    }
  }
}

The above source code contains three functions:

- HeatEqn_Initialize(CCTK_ARGUMENTS)
- HeatEqn_Update(CCTK_ARGUMENTS)
- HeatEqn_Boundary(CCTK_ARGUMENTS)

We also need a Makefile for our thorn, which is always called `make.code.defn`. This file tells Cactus the names of the source code files to compile. The following cell provides the content for the Makefile. Notice in our case we have three source code files to list: `init.c`, `evolve.c`, and `boundary.c`.

In [ ]:
%%writefile {thorn_dir}/src/make.code.defn
## Main make.code.defn file for thorn HeatEqn

## Source files in this directory
SRCS = init.c evolve.c boundary.c

## Subdirectories containing source files (none, in our case)
SUBDIRS =

## 3. Update thornlist

Before we can use the thorn, we need to add it to the thornlist and then rebuild Cactus.

In [ ]:
%cd {cactus_dir}

In [ ]:
thorn_list=os.path.join(os.getcwd(), 'configs', 'sim', 'ThornList')
thorn_list

In [ ]:
new_thorn_path=os.path.join(os.environ['ARR'], os.environ['THORN'])
new_thorn_path

In [ ]:
# Make sure the new thorn is in our ThornList
def add_thorn(new_thorn_path):
    contents = None
    with open(thorn_list, 'r') as f:
        contents = f.read()
    if new_thorn_path not in contents:
        with open(thorn_list, 'a') as f:
            f.write(new_thorn_path + '\n')
add_thorn(new_thorn_path)
!tail {thorn_list}

## 4. Rebuild Cactus

Now rebuild cactus. If all goes well, you will notice Cactus compiling your new thorn.

In [ ]:
%cd {cactus_dir}
!time ./simfactory/bin/sim build -j2

## 5. Create parfile

We're almost done! The last thing to do before running a simulation is to create a parfile. The parfile specifies which thorns we want to activate for our simulation, as well as parameter definitions for those thorns.

In the next cell we create the parfile, `heat_eqn.par`, in the parfile directory that we previously specified. Even though we did not declare any parameters in `params.ccl`, the thorns we activate do have parameters we need to define.

In [ ]:
%%writefile {par_dir}/heat_eqn.par

## Parameter definitions take the form of `ThornName::par_name = value`
Cactus::cctk_run_title = "scalar"
Cactus::cctk_full_warnings         = yes
Cactus::highlight_warning_messages = no
Cactus::terminate       = "time"
Cactus::cctk_final_time = 5.0

## ActiveThorns has an interesting syntax. Unlike with parameter definitions, the equals-sign should be read as "append".
## Subsequent `ActiveThorns = ...` assignments will not overwrite this one.
ActiveThorns = "Carpet CarpetLib CarpetInterp CarpetReduce CarpetSlab"
Carpet::verbose           = no
Carpet::veryverbose       = no
Carpet::schedule_barriers = no
Carpet::storage_verbose   = no
#Carpet::timers_verbose    = no
CarpetLib::output_bboxes  = no

Carpet::domain_from_coordbase = yes
Carpet::max_refinement_levels = 1

Carpet::ghost_size       = 1

Carpet::init_fill_timelevels = yes

ActiveThorns = "NaNChecker"
NaNChecker::check_every     = 1 # 512
#NaNChecker::verbose         = "all"
#NaNChecker::action_if_found = "just warn"
NaNChecker::action_if_found = "terminate"
NaNChecker::check_vars      = "
        HeatEqn::U
"

ActiveThorns = "Boundary CartGrid3D CoordBase SymBase"

# The following parameters are documented in the param.ccl
# file of cactusbase/CoordBase.
CoordBase::domainsize = "minmax"
CoordBase::spacing    = "numcells"

CoordBase::xmin = -10.0
CoordBase::ymin = -10.0
CoordBase::zmin = -10.0
CoordBase::xmax = +10.0
CoordBase::ymax = +10.0
CoordBase::zmax = +10.0
CoordBase::ncells_x  = 40
CoordBase::ncells_y  = 40
CoordBase::ncells_z  = 40

CoordBase::boundary_size_x_lower     = 1
CoordBase::boundary_size_y_lower     = 1
CoordBase::boundary_size_z_lower     = 1
CoordBase::boundary_size_x_upper     = 1
CoordBase::boundary_size_y_upper     = 1
CoordBase::boundary_size_z_upper     = 1

CartGrid3D::type = "coordbase"

ActiveThorns = "Time"

# dt = min(dx,dy,dz)*(Time::dtfac)
Time::dtfac = 0.0125

ActiveThorns = "InitBase"

## We must remember to activate our own thorn!
ActiveThorns = "HeatEqn"

ActiveThorns = "IOUtil"
IO::out_dir = $parfile
IO::checkpoint_dir                  = $parfile
IO::checkpoint_ID                   = no
IO::checkpoint_every_walltime_hours = 6.0
IO::checkpoint_on_terminate         = yes
IO::recover     = "autoprobe"
IO::recover_dir = $parfile
        
ActiveThorns="CarpetIOHDF5"
IOHDF5::out2D_every            = 16
IOHDF5::out2D_xz               = no
IOHDF5::out2D_yz               = no
IOHDF5::output_buffer_points   = yes
#IOHDF5::one_file_per_group     = yes
IOHDF5::output_symmetry_points = no
IOHDF5::compression_level      = 1
IOHDF5::use_checksums          = yes

# The double quotes delimit a multiline string.
IOHDF5::out2D_vars             = "
        HeatEqn::U
        Grid::Coordinates{out_every=1000000000} #we only want this to print once
"
IOHDF5::checkpoint             = no

## 6. Run simulation

Finally, we are ready to run the simulation using our new thorn. In the next cell, indicate the path to the directory where simulation output is written on your machine.

In [ ]:
## Linux example:
#sims_dir = '/home/ejwest/ETK/simulations'

## Windows example:
#sims_dir='C:\\Users\\EJWest\\ETK\\Simulations'

## fill in the path to your simulations directory
sims_dir=os.path.join(home_dir, "simulations") 

Remove any old versions of the simulation.

In [ ]:
rm -rf {sims_dir}/heat_eqn

Now run the simulation.

In [ ]:
%cd {cactus_dir}
!time ./simfactory/bin/sim create-run heat_eqn --parfile={par_dir}/heat_eqn.par --procs=1

## 7. Visualize output

The output of this simulation are `.h5` files, which are HDF5 files. HDF5 (Hierarchical Data Format 5) is a portable binary data format. As such, it is far more efficient to read and write than ASCII formats, and it is probably what you should normally use for visualizing large simulations. At this point you could switch over to VisIt in order to visualize the output. However it is also possible to use a Jupyter notebook using the H5Py library, which is a Python library that provides methods for interfacing with HDF5 files. Once the data are extracted from the `.h5` files, they can be visualized as normal using matplotlib.

### Extract data from hdf5 files

Let's first move to the output directory. 

In [ ]:
%cd {sims_dir}/heat_eqn/output-0000/heat_eqn

And then list the available `.h5` files.

In [ ]:
%ls *.h5

We will extract data from `U.xy.h5` into the notebook, specifically into NumPy arrays. To do this, we need to import NumPy and H5Py.

In [ ]:
## import libraries
import numpy as np
import h5py

In [ ]:
## create file handle
u_h5 = h5py.File("U.xy.h5","r")

The following cell extracts the time steps from the metadata of the HDF5 file.

In [ ]:
## get time steps
time_steps = np.array([])
for nm in u_h5:
    if hasattr(u_h5[nm], 'shape'):
        ts = u_h5[nm].attrs['timestep']
        time_steps = np.append(time_steps, ts)
    time_steps = np.sort(time_steps)

## print to screen (optional)
print("time_steps = ", time_steps)

In order to plot using `plot_surface()`, we need to construct an xy-meshgrid. This information is not stored in the `U.xy.h5` file, but instead is found in `x.xy.h5` and `y.xy.h5`. The following cell retrieves this information.

In [ ]:
## get xdata, ydata
x_h5 = h5py.File("x.xy.h5", "r")
y_h5 = h5py.File("y.xy.h5", "r")

for nm in x_h5:
    if hasattr(x_h5[nm], 'shape'):
        xdata = x_h5[nm][...]
for nm in y_h5:
    if hasattr(y_h5[nm], 'shape'):
        ydata = y_h5[nm][...]

## print to screen (optional)
print("xdata = ", xdata)
print("ydata = ", ydata)

The following cell extracts the values of U over the xy-grid at each time step.

In [ ]:
## get zdata
zdata = np.zeros((len(time_steps), xdata.shape[0], xdata.shape[1]))
for i in range(len(time_steps)):
    tstep = time_steps[i]
    ## find dataset with matching time step
    for nm in u_h5:
        if hasattr(u_h5[nm], 'shape'):
            if u_h5[nm].attrs['timestep'] == tstep: 
                ## save dataset as element of numpy array
                zdata[i,:,:] = u_h5[nm][...]

## print to screen (optional)
#print('zdata =',zdata)

### Plot using matplotlib's plot_surface( )

Now that the data we need are extracted from the HDF5 files, we can plot them as we normally would. Here we use matplotlib's `plot_surface()` command.

Plot a snapshot of the solution using `plot_surface()`.

In [ ]:
## set graphics backend
%matplotlib inline
## import graphics libraries
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm

## figure properties
cmap = cm.gist_rainbow
fig_width = 8
fig_height = 4
minval = np.min(zdata)
maxval = np.max(zdata)

## choose a time step
tstep = 1

## plot
fig = plt.figure(figsize=(fig_width, fig_height))
time = time_steps[tstep]
fig.suptitle('time = %.3f' % time)
ax = fig.add_subplot(111, projection='3d')
ax.set_zlim(minval, maxval)
xi = xdata
yi = ydata
zi = zdata[tstep,:,:]
ax.plot_surface(xi, yi, zi, cstride=1, rstride=1, cmap=cmap, clim=(minval, maxval))
plt.show()

Put all time steps together into an animation using matplotlib's `animation.FuncAnimation()` command.

In [ ]:
## set graphics backend
%matplotlib inline
## import graphics libraries
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.cm as cm
import matplotlib.animation as animation

## figure properties
cmap = cm.gist_rainbow
fig_width = 8
fig_height = 4
minval = np.min(zdata)
maxval = np.max(zdata)

## initialize plot
plt.ioff()
fig = plt.figure(figsize=(fig_width, fig_height))
time = time_steps[0]
fig.suptitle('time = %.3f' % time)
ax = fig.add_subplot(111, projection='3d')
ax.set_zlim(minval, maxval)
xi = xdata
yi = ydata
zi = zdata[0,:,:]
scene = [ax.plot_surface(xi, yi, zi, cstride=1, rstride=1, cmap=cmap, clim=(minval, maxval))]

## update plot
def update_plot(frame_number, time_steps, xdata, ydata, zdata, scene):
    time = time_steps[frame_number]
    fig.suptitle('time = %.3f' % time)
    scene[0].remove()
    xi = xdata
    yi = ydata
    zi = zdata[frame_number,:,:]
    scene[0] = ax.plot_surface(xi, yi, zi, cstride=1, rstride=1, cmap=cmap, clim=(minval, maxval))

## make animation
fps=5
frames=np.arange(0, len(time_steps), 1)
anim = animation.FuncAnimation(
    fig, update_plot, frames, fargs=(time_steps, xdata, ydata, zdata, scene), 
    interval=1000/fps, blit=False, repeat=False)
plt.close()

## play animation
from IPython.display import HTML
HTML(anim.to_html5_video())  #playback option 1
#HTML(anim.to_jshtml())       #playback option 2

Save the animation to a file, if you like. In order to save as an `.mp4` file, you need to have the ffmpeg library installed on your machine. 

In [ ]:
## save animation to file
file = 'heat_eqn'
#anim.save(file + '.mp4', writer='ffmpeg', fps=fps)      # requires ffmpeg
anim.save(file + '.gif', writer='imagemagick', fps=fps)

In [ ]:
## Display the gif file by uncommenting these lines
# from IPython.display import Image
# display(Image(file+".gif"))

## 8. Exercises

1. Change the time stepsize and termination time of the simulation.
2. Change the grid size and grid spacing of the simulation.
3. Change the height/width/location of the initial pulse.
4. Change the boundary conditions: Neumann, mixed, periodic.
5. Add a parameter to allow selecting the boundary condition.
6. Add a diffusion parameter and run for different values.

## Acknowledgments

This notebook grew out of a series of notebook-based tutorials delivered by Steve Brandt at the 2017 Einstein Toolkit school and workshop at NCSA. Those tutorials were based on his "Funwave" example--a simulation that models coastal waves. The current notebook is an attempt at a condensed version of the "create a new thorn" tutorial for new user's. To that end, the Funwave example has been replaced by a simpler case study--the standard heat equation--in order to fit everything into a single notebook. The aim has been to provide a more transparent example of how to create a new thorn from scratch. For new users, this is a daunting task, which the author of this notebook fully recognizes.

A big thanks goes to Steve Brandt for his outstanding series of Jupyter notebooks on the in's and out's of the Einstein Toolkit. His tutorials go far beyond this one, both in depth and breadth. They cover many common gotcha issues that this notebook glosses over. I would encourage any new user to work through them; it is well worth the effort to get them to work on your machine. They can be found in the Google drive [here](https://drive.google.com/drive/folders/0B4gNfWainf-5R3BldGJuRlo4dHc) and video of the 2017 workshop sessions can be found on YouTube [here](https://www.youtube.com/watch?v=jqgEaUjl23I), starting at the 4:08:37 mark.

Thanks also goes to Yosef Zlochower, whose heat equation example has been used in this notebook.

My role has basically been to merge the pedagogical clarity and thoroughness of Steve's tutorials (including his fully commented ccl file templates) with the simplicity of Yosef's example. I am fully responsible for any bugs or confusions that remain in the final result. --Eric J West

Some edits and additions made in April 2024 by Max Morris.